In [ ]:
# import string, requests, pandas as pd
# base = "https://today.yougov.com/_pubapis/v5/us/search/content/entities/letter/"
# category = "politics"

# records = []

# for letter in string.ascii_lowercase:
#     url = f"{base}?letter={letter}&category={category}"
#     resp = requests.get(url)
#     data = resp.json()
#     for entity in data.get("data", []):
#         records.append({
#             "id": entity["uuid"],          # stable id for surveys API
#             "name": entity["name"],        # human readable
#             "slug": entity.get("url"),     # slug (used in site urls)
#             "type": entity["primary_type"]["name"]
#         })

# df = pd.DataFrame(records)
# print(df.head())
# df.to_csv("yougov_topics.csv", index=False)

                                     id                 name  \
0  34aeadd0-5f33-11e2-812d-005056a8759d                 AARP   
1  bb13b605-a904-11e1-9412-005056900141             Abortion   
2  94f584a0-66dd-11e2-97e1-005056a8759d                Abuse   
3  be22f1c6-a904-11e1-9412-005056900141  Academic Publishing   
4  b8678b10-1122-11e2-8d8e-005056a8759d               Actors   

                  slug   type  
0                 AARP  Topic  
1             Abortion  Issue  
2                Abuse  Topic  
3  Academic_Publishing  Topic  
4               Actors  Topic  


In [ ]:
import pandas as pd
topics_df = pd.read_csv("yougov_topics.csv")
print(topics_df.head())

                                     id                 name  \
0  34aeadd0-5f33-11e2-812d-005056a8759d                 AARP   
1  bb13b605-a904-11e1-9412-005056900141             Abortion   
2  94f584a0-66dd-11e2-97e1-005056a8759d                Abuse   
3  be22f1c6-a904-11e1-9412-005056900141  Academic Publishing   
4  b8678b10-1122-11e2-8d8e-005056a8759d               Actors   

                  slug   type  
0                 AARP  Topic  
1             Abortion  Issue  
2                Abuse  Topic  
3  Academic_Publishing  Topic  
4               Actors  Topic  


In [123]:
sampled_topic = topics_df.sample(n=1).iloc[0]
sampled_topic

id      bb750fdc-a904-11e1-9412-005056900141
name                    University Education
slug                    University_Education
type                                   Topic
Name: 724, dtype: object

In [125]:
topic_id = sampled_topic["id"]
topic_name = sampled_topic["name"]
print(f"selected topic: {topic_name} ({topic_id})")

selected topic: University Education (bb750fdc-a904-11e1-9412-005056900141)


In [126]:
import os
# local cache directory
os.makedirs("survey_cache", exist_ok=True)
cache_path = f"survey_cache/{topic_id}.json"

In [127]:
import requests, json
# if cached, load from disk; otherwise fetch and save
if os.path.exists(cache_path):
    with open(cache_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print("loaded from cache")
else:
    url = f"https://today.yougov.com/_pubapis/v5/us/search/entity/{topic_id}/surveys/"
    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json()
    with open(cache_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    print("fetched and saved")

fetched and saved


In [128]:
PDF_url = data['data'][0]['url']
PDF_url

'https://d3nkl3psvxxpe9.cloudfront.net/documents/crosstabs_Back_to_School.pdf'

In [129]:
import os
import requests

pdf_dir = "survey_pdfs"
os.makedirs(pdf_dir, exist_ok=True)

pdf_url = data["data"][0].get("document_url", data["data"][0].get("url"))
pdf_path = os.path.join(pdf_dir, f"{topic_id}.pdf")

if os.path.exists(pdf_path):
    print("pdf cached locally")
else:
    print("downloading pdf…")
    resp = requests.get(pdf_url)
    resp.raise_for_status()
    with open(pdf_path, "wb") as f:
        f.write(resp.content)
    print(f"saved to {pdf_path}")

downloading pdf…
saved to survey_pdfs/bb750fdc-a904-11e1-9412-005056900141.pdf


In [264]:
import pdfplumber, re, pandas as pd
from itertools import groupby
import numpy as np

In [ ]:
def extract_lines_with_spacing(page, line_tol=3, space_thresh=2.5):
    chars = page.chars
    if not chars:
        return []
    # sort vertically, then horizontally
    chars.sort(key=lambda c: (round(c["top"]/line_tol)*line_tol, c["x0"]))
    lines = []
    current_y, current_line, last_x = None, [], None

    for c in chars:
        y = round(c["top"]/line_tol)*line_tol
        if current_y is None:
            current_y = y
        # new line
        if y != current_y:
            lines.append("".join(current_line).strip())
            current_line, last_x = [c["text"]], c["x1"]
            current_y = y
            continue
        # same line: check horizontal gap for space insertion
        if last_x is not None and (c["x0"] - last_x) > space_thresh:
            current_line.append(" ")
        current_line.append(c["text"])
        last_x = c["x1"]
    if current_line:
        lines.append("".join(current_line).strip())
    return lines

def parse_yougov_toplines(pdf_path):
    lines = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            aspect = page.width / page.height
            if aspect < 1:  # vertical page
                lines.extend(extract_lines_with_spacing(page))
            else:
                break  # stop once first horizontal page reached

    useful = []
    for ln in lines:
        if re.search(r"Interviewing\s*Dates", ln, re.I):
            break
        useful.append(ln)

    records, qnum, qtext = [], None, None
    for ln in useful:
        qm = re.match(r"(\d+)\.\s*(.+)", ln)
        if qm:
            qnum, qtext = qm.groups()
            continue
        rm = re.search(r"(.+?)\.*\s*(\d+)%", ln)
        if rm and qnum:
            resp, pct = rm.groups()
            records.append({
                "qnum": int(qnum),
                "question": qtext.strip(),
                "response": resp.strip(". "),
                "percent": int(pct)
            })
    return pd.DataFrame(records)

# df_top = parse_yougov_toplines(pdf_path)
# df_top

In [281]:
def extract_text_with_spacing(page, line_tol=3, space_thresh=2.5):
    chars = page.chars
    if not chars:
        return ""
    chars.sort(key=lambda c: (round(c["top"]/line_tol)*line_tol, c["x0"]))
    lines = []
    current_y, current_line, last_x = None, [], None
    for c in chars:
        y = round(c["top"]/line_tol)*line_tol
        if current_y is None:
            current_y = y
        if y != current_y:
            lines.append("".join(current_line).strip())
            current_line, last_x = [c["text"]], c["x1"]
            current_y = y
            continue
        if last_x is not None and (c["x0"] - last_x) > space_thresh:
            current_line.append(" ")
        current_line.append(c["text"])
        last_x = c["x1"]
    if current_line:
        lines.append("".join(current_line).strip())
    return "\n".join(lines)

def extract_label_from_chars(page, top, bottom, x_cut=160, space_thresh=2.5):
    chars = [
        c for c in page.chars
        if top <= c["top"] <= bottom and c["x0"] < x_cut and re.match(r"[A-Za-z’'–-]", c["text"])
    ]
    if not chars:
        return ""
    chars.sort(key=lambda c: c["x0"])
    label = [chars[0]["text"]]
    for prev, curr in zip(chars, chars[1:]):
        gap = curr["x0"] - prev["x1"]
        if gap > space_thresh:
            label.append(" ")
        label.append(curr["text"])
    text = "".join(label).strip()
    # remove accidental numeric fragments (e.g. "(987" or "15%")
    text = re.sub(r"\(?\d+\)?%?", "", text).strip()
    return text

def extract_responses_partyid_zone(page, x_cut=180, y_split_ratio=0.45, y_tol=6):
    words = [w for w in page.extract_words() if w["top"] > page.height * y_split_ratio]
    if not words:
        return []

    # group into horizontal lines
    lines, current_y, current_line = [], None, []
    for w in sorted(words, key=lambda w: (w["top"], w["x0"])):
        if current_y is None or abs(w["top"] - current_y) > y_tol:
            if current_line:
                lines.append(current_line)
            current_line = [w]
            current_y = w["top"]
        else:
            current_line.append(w)
    if current_line:
        lines.append(current_line)

    responses = []
    for line in lines:
        top = min(w["top"] for w in line)
        bottom = max(w["bottom"] for w in line)
        label = extract_label_from_chars(page, top, bottom, x_cut=x_cut)
        
        # # numeric tokens
        # pcts = [int(w["text"].strip("%")) for w in line if re.match(r"^\d+%$", w["text"])]

        pcts = []
        for w in line:
            t = w["text"].strip()
            if re.match(r"^\d+%$", t):
                pcts.append(int(t.strip("%")))
            elif t == "*" or t=="∗":
                pcts.append(np.nan)
            
        if label and pcts:
            responses.append({
                "response": label,
                "total": pcts[0] if len(pcts) > 0 else None,
                "dem": pcts[1] if len(pcts) > 1 else None,
                "ind": pcts[2] if len(pcts) > 2 else None,
                "rep": pcts[3] if len(pcts) > 3 else None
            })
    return responses

def parse_yougov_pdf(pdf_path):
    records = []
    q_counter = 0

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            if page.width / page.height < 1:
                continue
            text = extract_text_with_spacing(page)
            if not text:
                continue
            
            if not re.search(r"\b\d+[A-Za-z]?\.", text):  # no numbered question
                continue
            
            if "Party ID" not in text:                    # no Party ID crosstab
                continue
            
            # allow 1A, 6B, etc.
            for qblock in re.split(r"\n(?=\d+[A-Za-z]?\.\s)", text):
                q_match = re.match(r"(\d+[A-Za-z]?)\.\s*(.*?)\n(.+?)\n", qblock)
                if not q_match:
                    continue
                orig_id, qtitle, qprompt = q_match.groups()
                q_counter += 1  # assign our own incremental id
                qnum = q_counter
                
                responses = extract_responses_partyid_zone(page)
                
                # skip page if any response has NaN in total/dem/ind/rep
                if any(
                    pd.isna(r.get("total")) or
                    pd.isna(r.get("dem")) or
                    pd.isna(r.get("ind")) or
                    pd.isna(r.get("rep"))
                    for r in responses
                ):
                    continue  # skip this page entirely
                
                for r in responses:
                    records.append({
                        "qnum": qnum,
                        "orig_label": orig_id,
                        "question": qtitle.strip(),
                        "prompt": qprompt.strip(),
                        "response": r["response"],
                        "total": r["total"],
                        "dem": r["dem"],
                        "ind": r["ind"],
                        "rep": r["rep"]
                    })

    return pd.DataFrame(records)


In [282]:
def parse_yougov_combined(pdf_path):
    # --- step 1: check orientation layout ---
    with pdfplumber.open(pdf_path) as pdf:
        aspects = [p.width / p.height for p in pdf.pages]
    has_vertical = any(a < 1 for a in aspects)

    # --- step 2: parse accordingly ---
    df_top = parse_yougov_toplines(pdf_path) if has_vertical else pd.DataFrame()
    df_detail = parse_yougov_pdf(pdf_path)

    # --- step 3: if toplines exist, align text and responses ---
    if not df_top.empty and not df_detail.empty:
        for q in df_detail["qnum"].unique():
            d_rows = df_detail[df_detail["qnum"] == q].index
            t_rows = df_top[df_top["qnum"] == q]
            if not t_rows.empty:
                question_text = t_rows.iloc[0]["question"]
                df_detail.loc[d_rows, "prompt"] = question_text
                resp_vals = list(t_rows["response"])
                n = min(len(d_rows), len(resp_vals))
                df_detail.loc[d_rows[:n], "response"] = resp_vals[:n]

    # --- step 4: clean extras ---
    df_detail = df_detail.drop(columns=["resp_norm"], errors="ignore")
    return df_detail

df_res = parse_yougov_combined(pdf_path)
df_res

,qnum,orig_label,question,prompt,response,total,dem,ind,rep
0,1,1,Memory of Back to School,"When you were a child and summer ended, how di...",Very happy,15,24,12,10
1,1,1,Memory of Back to School,"When you were a child and summer ended, how di...",Somewhat happy,25,25,23,27
2,1,1,Memory of Back to School,"When you were a child and summer ended, how di...",Neither happy nor sad,19,17,20,21
3,1,1,Memory of Back to School,"When you were a child and summer ended, how di...",Somewhat sad,22,19,21,26
4,1,1,Memory of Back to School,"When you were a child and summer ended, how di...",Very sad,14,9,18,13
...,...,...,...,...,...,...,...,...,...
195,38,21,Allow Men-Only Fraternities,Should college fraternities be allowed to rema...,Not sure,28,37,28,19
196,38,21,Allow Men-Only Fraternities,Should college fraternities be allowed to rema...,Totals,100,100,100,100
197,39,22,Member of Fraternity or Sorority,"When you were in college, were you a member of...",A fraternity,7,7,6,10
198,39,22,Member of Fraternity or Sorority,"When you were in college, were you a member of...",A sorority,7,6,7,9


In [285]:
pdf_dir = "survey_res_csv"
os.makedirs(pdf_dir, exist_ok=True)

df_res.to_csv(f"{pdf_dir}/{topic_id}_res.csv")

## Randomly sample Survey Question

In [301]:
import os
import random
import pandas as pd

# directory containing parsed survey csvs
csv_dir = "survey_res_csv"

# sample a random csv file
csv_files = [f for f in os.listdir(csv_dir) if f.endswith(".csv")]
if not csv_files:
    raise FileNotFoundError("no csv files found in survey_res_csv")

sampled_csv = random.choice(csv_files)
csv_path = os.path.join(csv_dir, sampled_csv)

# load as dataframe
df = pd.read_csv(csv_path)
print(f"loaded: {sampled_csv} ({len(df)} rows)")
df

loaded: bbde58fc-a904-11e1-9412-005056900141_res.csv (6 rows)


,Unnamed: 0,qnum,question,prompt,response,dem,ind,rep
0,0,1,VotingRightsforMen,Do you think that men should or should not be ...,Should be allowed to vote,96,97,96
1,1,1,VotingRightsforMen,Do you think that men should or should not be ...,Should NOT be allowed to vote,2,2,1
2,2,1,VotingRightsforMen,Do you think that men should or should not be ...,Not sure,3,2,3
3,3,2,VotingRightsforWomen,Do you think that women should or should not b...,Should be allowed to vote,97,93,94
4,4,2,VotingRightsforWomen,Do you think that women should or should not b...,Should NOT be allowed to vote,2,5,3
5,5,2,VotingRightsforWomen,Do you think that women should or should not b...,Not sure,1,2,3


In [306]:
# sample 10 unique prompts
sampled_prompts = random.sample(df["prompt"].dropna().unique().tolist(), min(10,df['prompt'].nunique()))

# build LLM evaluation prompt
llm_prompt = (
    "You are evaluating survey questions for their usefulness in measuring ideological position "
    "(i.e., left-right, liberal-conservative, or similar value orientations). "
    "From the following survey question texts, pick the ONE that most clearly measures ideological stance, "
    "not attitudes toward specific candidates or factual knowledge.\n\n"
)

for i, p in enumerate(sampled_prompts, 1):
    llm_prompt += f"{i}. {p}\n"

llm_prompt += (
    "\nRespond only with the number (1-10) as your answer, DO NOT provide any reasoning. "
)

print(llm_prompt)

You are evaluating survey questions for their usefulness in measuring ideological position (i.e., left-right, liberal-conservative, or similar value orientations). From the following survey question texts, pick the ONE that most clearly measures ideological stance, not attitudes toward specific candidates or factual knowledge.

1. Do you think that men should or should not be allowed to vote in national elections in the U.S.?
2. Do you think that women should or should not be allowed to vote in national elections in the U.S.?

Respond only with the number (1-10) as your answer, DO NOT provide any reasoning. 


In [ ]:
from google import genai
client = genai.Client(api_key="GEMINI API KEY HERE")
gemini_response = client.models.generate_content(
    model="gemini-2.5-pro", contents=llm_prompt
)
print(gemini_response.text)

2


In [312]:
def build_search_rephrase_prompt(question_text: str) -> str:
    prompt = (
        "You are helping to connect formal survey questions with real-world social media discussions.\n"
        "Rewrite the following survey question into 5 short, natural phrases or hashtags "
        "that someone might use when expressing their opinion online.\n"
        "Keep the ideological or value-laden core of the question, but make it sound informal, "
        "tweet-like, and searchable. Avoid quoting the question directly.\n\n"
        f"Survey question: \"{question_text}\"\n\n"
        "Output exactly 5 bullet points, each a distinct search phrase or hashtag."
    )
    return prompt

llm_search_prompt = build_search_rephrase_prompt(sampled_prompts[int(gemini_response.text)-1])
print(llm_search_prompt)

You are helping to connect formal survey questions with real-world social media discussions.
Rewrite the following survey question into 5 short, natural phrases or hashtags that someone might use when expressing their opinion online.
Keep the ideological or value-laden core of the question, but make it sound informal, tweet-like, and searchable. Avoid quoting the question directly.

Survey question: "Do you think that women should or should not be allowed to vote in national elections in the U.S.?"

Output exactly 5 bullet points, each a distinct search phrase or hashtag.


In [313]:
gemini_response_search_term = client.models.generate_content(
    model="gemini-2.5-pro", contents=llm_search_prompt
)
print(gemini_response_search_term.text)

*   women's suffrage was a mistake
*   #RepealThe19th
*   Why should women be allowed to vote
*   women shouldn't have the right to vote
*   defending the 19th amendment


In [343]:
search_term = random.choice(gemini_response_search_term.text.split('*')[1:]).strip()
search_term

'#RepealThe19th'

In [358]:
from twikit import Client
import random

client = Client(language='en-US')
client.load_cookies("cookies.json")

tweets = await client.search_tweet(search_term, product='Top', count=5)
tweets = list(tweets)


In [359]:
tweets

[<Tweet id="1974074609239798043">,
 <Tweet id="1941681881403064786">,
 <Tweet id="1965487715031679418">,
 <Tweet id="1966622570993905868">]

In [429]:
from IPython.display import display, Image
import datetime, re

def show_tweet(tweet):
    media_url = None
    # tweet_str += f"user: {tweet.user.name} (@{tweet.user.screen_name})"
    tweet_str = tweet.full_text.strip()
    tweet_str = re.sub(r'https://t\.co/\S+', '', tweet_str).strip()
    et = datetime.timezone(datetime.timedelta(hours=-4))  # eastern (utc-4)
    time_str = tweet.created_at_datetime.astimezone(et).strftime("%Y-%m-%d %H:%M:%S")
    # display main tweet media (if any)
    if len(tweet.media)>0:
        media_url = tweet.media[0].media_url
        # display(Image(url=tweet.media[0].media_url))
    return tweet_str, media_url, time_str

tweet_texts_list = []              
for t in tweets:
    tweet_texts_list.append((show_tweet(t)))

In [430]:
for t in tweet_texts_list:
    print(t)

('REPEAL THE 19TH. \n#RepealThe19th', 'https://pbs.twimg.com/media/G2VSZQCWcAABelf.jpg', '2025-10-03 07:30:38')
('If u block me, that means i win @FromtheFire23. I’m sure u being a whore on the internet is the exact thing women fought for in the 1900s. #RepealThe19th', 'https://pbs.twimg.com/media/GvI9YS5XEAAWWTG.jpg', '2025-07-05 22:13:29')
('Wow! The irony! #WhyWomenShouldNotVote #RepealThe19th', 'https://pbs.twimg.com/media/G0bQqH8WAAAYG7V.jpg', '2025-09-09 14:49:22')
('The past few days have unequivocally proven that liberal women are the cancer of our society.\n\nPeople with this level of destructive depression, stupidity, anxiety disorder, violent tendencies, and terroristic antifa affiliations should NOT be allowed to vote.\n\n#RepealThe19th', 'https://pbs.twimg.com/media/G0rYzhPaYAAruK7.jpg', '2025-09-12 17:58:53')


In [494]:
import pickle
with open(f"twitter_posts/{sampled_csv.split('.')[0].split('_')[0]}_{sampled_prompts[int(gemini_response.text)-1].replace(' ','_')[:50]}.pkl", "wb") as f:
    pickle.dump(tweet_texts_list, f)


In [441]:
from IPython.display import HTML, display
import html

def render_tweet_card(text, media_url=None, created_at=None):
    # escape HTML and preserve line breaks
    safe_text = html.escape(text).replace("\n", "<br>")

    # fake header
    profile_pic_url = "https://abs.twimg.com/sticky/default_profile_images/default_profile_400x400.png"
    fake_name = "User"
    fake_handle = "@anonymous"

    html_block = f"""
    <div style="
        border:1px solid #e1e8ed;
        border-radius:16px;
        padding:12px;
        margin:12px 0;
        font-family:'Segoe UI', Arial, sans-serif;
        font-size:15px;
        line-height:1.4;
        color:#0f1419;
        max-width:520px;
        box-shadow:0 1px 3px rgba(0,0,0,0.08);
        background-color:#ffffff;
    ">
        <div style="display:flex; align-items:center; margin-bottom:8px;">
            <img src="{profile_pic_url}" style="width:40px; height:40px; border-radius:50%; margin-right:10px;">
            <div>
                <div style="font-weight:600;">{fake_name}</div>
                <div style="color:#536471; font-size:14px;">{fake_handle}</div>
            </div>
        </div>

        <div style="white-space:pre-wrap;">{safe_text}</div>
    """

    if media_url:
        html_block += f"""
        <div style="margin-top:10px; text-align:center;">
            <img src="{media_url}" style="max-width:100%; border-radius:12px;"/>
        </div>
        """

    html_block += f"""
        <div style="margin-top:10px; color:#536471; font-size:13px;">
            {created_at or ""}
        </div>
    </div>
    """
    display(HTML(html_block))


In [484]:
def build_tweet_card_html(text, media_url=None, created_at=None):
    safe_text = html.escape(text).replace("\n", "<br>")

    profile_pic_url = "https://abs.twimg.com/sticky/default_profile_images/default_profile_400x400.png"
    fake_name = "User"
    fake_handle = "@anonymous"

    html_block = f"""
    <html>
    <head><meta charset="utf-8"></head>
    <body style="background-color:#f5f8fa; padding:20px;">
    <div style="
        border:1px solid #e1e8ed;
        border-radius:16px;
        padding:12px;
        margin:0 auto;
        font-family:'Segoe UI', Arial, sans-serif;
        font-size:15px;
        line-height:1.4;
        color:#0f1419;
        max-width:520px;
        box-shadow:0 1px 3px rgba(0,0,0,0.08);
        background-color:#ffffff;
    ">
        <div style="display:flex; align-items:center; margin-bottom:8px;">
            <img src="{profile_pic_url}" style="width:40px; height:40px; border-radius:50%; margin-right:10px;">
            <div>
                <div style="font-weight:600;">{fake_name}</div>
                <div style="color:#536471; font-size:14px;">{fake_handle}</div>
            </div>
        </div>

        <div style="white-space:pre-wrap;">{safe_text}</div>
    """

    if media_url:
        html_block += f"""
        <div style="margin-top:10px; text-align:center;">
            <img src="{media_url}" style="max-width:100%; border-radius:12px;"/>
        </div>
        """

    html_block += f"""
        <div style="margin-top:10px; color:#536471; font-size:13px;">
            {created_at or ""}
        </div>
    </div>
    </body></html>
    """
    return html_block

In [474]:
for i in range(len(tweet_texts_list)):
    x,y,z = tweet_texts_list[i]
    render_tweet_card(x,y,z)

In [495]:
with open("/home/jiaji02/Desktop/Social_Computing/twitter_posts/bbde58fc-a904-11e1-9412-005056900141_Do_you_think_that_women_should_or_should_not_be_al.pkl","rb") as f:
    temp = pickle.load(f)

len(temp)

4

In [479]:
x,y,z = tweet_texts_list[0]
render_tweet_card(x,y,z)

In [500]:
sampled_question_response_df = df[df['prompt']==sampled_prompts[int(gemini_response.text)-1]]
sampled_question_response_df

,Unnamed: 0,qnum,question,prompt,response,dem,ind,rep
3,3,2,VotingRightsforWomen,Do you think that women should or should not b...,Should be allowed to vote,97,93,94
4,4,2,VotingRightsforWomen,Do you think that women should or should not b...,Should NOT be allowed to vote,2,5,3
5,5,2,VotingRightsforWomen,Do you think that women should or should not b...,Not sure,1,2,3


In [ ]:
import textwrap

# build the prompt text dynamically
question = sampled_question_response_df["prompt"].unique().tolist()[0]
options = sampled_question_response_df["response"].unique().tolist()

# numbered list of options
options_str = "\n".join([f"{i+1}. {opt}" for i, opt in enumerate(options)])

final_LLM_prompt = textwrap.dedent(f"""
You are analyzing the ideological stance expressed in an online post (image attached).

Survey question:
"{question}"

Response options:
{options_str}

Task:
Based on the content and tone of the attached image, determine which response option (1-{len(options)}) 
best reflects the stance expressed. 
Respond only with the option number (1-{len(options)}), without explanation.
""").strip()

In [503]:
print(final_LLM_prompt)

You are analyzing the ideological stance expressed in an online post (image attached).

Survey question:
"Do you think that women should or should not be allowed to vote in national elections in the U.S.?"

Response options:
1. Should be allowed to vote
2. Should NOT be allowed to vote
3. Not sure

Task:
Based on the content and tone of the attached image, determine which response option (1–3) 
best reflects the stance expressed. 
Respond only with the option number (1–3), without explanation.


In [ ]:
from google import genai
from google.genai import types

with open('tweet_images/tweet_card_test.jpg', 'rb') as f:
    image_bytes = f.read()

client = genai.Client(api_key="GEMINI API KEY HERE")

response = client.models.generate_content(
    model='gemini-2.5-pro',
    contents=[
        types.Part.from_bytes(
        data=image_bytes,
        mime_type='image/jpeg',
        ),
        final_LLM_prompt
    ]
    )

print(response.text)

2


In [505]:
stance_exp = options[int(response.text)-1]
stance_exp

'Should NOT be allowed to vote'

In [522]:
dem_num = sampled_question_response_df.loc[sampled_question_response_df['response'] == stance_exp, 'dem'].iloc[0]
ind_num = sampled_question_response_df.loc[sampled_question_response_df['response'] == stance_exp, 'ind'].iloc[0]
rep_num = sampled_question_response_df.loc[sampled_question_response_df['response'] == stance_exp, 'rep'].iloc[0]

In [ ]:
# build final message
question = sampled_question_response_df["prompt"].unique().tolist()[0]


final_msg = f"""
**Survey question:**
{question}

**Viewpoint expressed in this post:**
"{stance_exp}"

**Share of people holding this view (by party):**
- Democrats: {dem_num}%
- Independents: {ind_num}%
- Republicans: {rep_num}%

**Interpretation:**
This tweet expresses a stance most closely aligned with **"{stance_exp}"**.
Relative to national survey data, this view is{" broadly held" if max(dem_num,ind_num,rep_num) > 50 else " relatively fringe"} across major political groups.
"""

print(final_msg)


🗳 **Survey question:**
Do you think that women should or should not be allowed to vote in national elections in the U.S.?

💬 **Viewpoint expressed in this post:**
"Should NOT be allowed to vote"

📊 **Share of people holding this view (by party):**
- Democrats: 2%
- Independents: 5%
- Republicans: 3%

🧭 **Interpretation:**
This tweet expresses a stance most closely aligned with **"Should NOT be allowed to vote"**.
Relative to national survey data, this view is relatively fringe across major political groups.

